In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import langchain
langchain.debug = True

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index=True   # 分割された位置を保存できる
)

In [4]:
from langchain.document_loaders import YoutubeLoader
youtube_docs = YoutubeLoader.from_youtube_url(
   youtube_url="https://www.youtube.com/watch?v=X550Zbz_ROE",
   language="en"
).load_and_split(text_splitter)

In [5]:
persist_directory = "./tmp/chroma_db"

In [6]:
!rm -rf $persist_directory

In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
collection_name = "chroma_test"
db = Chroma.from_documents(youtube_docs, embeddings, collection_name=collection_name, persist_directory=persist_directory)

In [9]:
retriever = db.as_retriever(search_kwargs={"k": 1})

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0613")
chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

In [11]:
question = "LangChain Agentとはなんですか？"
res = chain(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "LangChain Agentとはなんですか？"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "LangChain Agentとはなんですか？",
  "context": "Context:\nthat do things like summarize the conversation as it goes along we've got other ones where we limit the window so it only remembers the last few encounters and then we've got sort of things where you know it merges some of those so where it has the last few but then it does a summary for everything else the summary though is actually done by calling out to a language model itself and asking it hey summarize this conversation so that's something that you will see as well then we've got some sort of more external ways of doing this so these can be things like making some kind of Knowledge Graph mem

In [12]:
print(res["result"])

LangChain Agentは、OpenAIのLangChainモデルを使用して対話を行うエージェントのことです。このエージェントは、会話の要約や過去のエンカウンターの記憶など、さまざまなメモリ機能を持っています。また、カスタマイズも可能で、独自のメモリシステムを使用することもできます。


In [13]:
follow_up_question =  "具体的な例を教えてください"
res = chain(follow_up_question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "具体的な例を教えてください"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "具体的な例を教えてください",
  "context": "Context:\njust print that out and we can look at it and we can see that yeah response is doing great when sem asks for customer support the AI responds positively and asks what kind of Customs support Sam needs so you'll notice here that it's kind of doing a co-reference resolution in that rather than a human would probably say it responds to what he asked or she here it's sticking to Ai and Sam in there so this is a summary one it's also very useful the next one is is kind of an alternate version of the first one that we had so this is a conversation buffer window memory so we're going to be doing the same sort of thing of j

In [14]:
res["result"]

'具体的な例として、以下のような会話が考えられます。\n\nユーザー: こんにちは、私はサムです。何かサポートが必要です。\nAI: 了解しました、どのようなサポートが必要でしょうか？\nユーザー: テレビが動かなくなってしまいました。\nAI: それは困りますね。具体的にどのような症状が見られますか？\nユーザー: 電源を入れると変な音がして画面が真っ暗になります。\n\nこの会話では、AIがユーザーの要件を理解し、適切な質問を返しています。AIは直前の2つの対話を参照して、より適切な応答を生成しています。'

## Conversational Retriveal Chain

In [15]:
from langchain.chains import ConversationalRetrievalChain

In [16]:
cr_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever
)

In [17]:
chat_history = []
res = cr_chain({"question": question, "chat_history": chat_history})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "LangChain Agentとはなんですか？",
  "chat_history": []
}
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "LangChain Agentとはなんですか？",
  "context": "that do things like summarize the conversation as it goes along we've got other ones where we limit the window so it only remembers the last few encounters and then we've got sort of things where you know it merges some of those so where it has the last few but then it does a summary for everything else the summary though is actually done by calling out to a language model itself and asking it hey summarize this conversation so that's something that you will see as well then we've got some sort of more external ways of doing this s

In [18]:
res["answer"]

'LangChain Agentは、OpenAIのLangChainフレームワークを使用して作成された会話モデルです。このエージェントは、対話の内容を記憶し、それを利用して追加の応答を生成することができます。さまざまな種類のメモリを使用して、会話の要約や追加の情報を提供することができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。'

In [19]:
chat_history.append((question, res["answer"]))
res = cr_chain({"question": follow_up_question, "chat_history": chat_history})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "具体的な例を教えてください",
  "chat_history": [
    [
      "LangChain Agentとはなんですか？",
      "LangChain Agentは、OpenAIのLangChainフレームワークを使用して作成された会話モデルです。このエージェントは、対話の内容を記憶し、それを利用して追加の応答を生成することができます。さまざまな種類のメモリを使用して、会話の要約や追加の情報を提供することができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。"
    ]
  ]
}
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "具体的な例を教えてください",
  "chat_history": "\nHuman: LangChain Agentとはなんですか？\nAssistant: LangChain Agentは、OpenAIのLangChainフレームワークを使用して作成された会話モデルです。このエージェントは、対話の内容を記憶し、それを利用して追加の応答を生成することができます。さまざまな種類のメモリを使用して、会話の要約や追加の情報を提供することができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。"
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Given the following conversation and a follow up question, rephrase the follow up question

In [20]:
res["answer"]

'はい、具体的な例をいくつかご紹介します。\n\n例1：\nAI: こんにちは、私はサムです。どうしたらお手伝いできますか？\nユーザー: こんにちは、私の壊れたテレビの修理に助けが必要です。\nAI: それは困りますね。具体的に何が問題なのでしょうか？\n（ここでAIは会話のバッファを保持し、会話のサマリーを更新します）\n\n例2：\nAI: こんにちは、私はサムです。どうしたらお手伝いできますか？\nユーザー: こんにちは、私の壊れたテレビの修理に助けが必要です。\nAI: それは困りますね。具体的に何が問題なのでしょうか？\nユーザー: おそらくハードウェアの問題です。\n（ここでAIは会話のバッファを保持し、会話のサマリーを更新します）\n\nこれらの例では、AIがユーザーとの会話を進めながら、会話のバッファを保持し、最新の会話のサマリーを提供しています。'

In [21]:
prompt_template = """
次の会話とフォローアップの質問がある場合、会話を考慮してフォローアップの質問を独立した質問に言い換えなさい。

会話:
{chat_history}
フォローアップの質問: {question}
独立した質問:"""

cr_chain.question_generator.prompt.template = prompt_template

In [22]:
res = cr_chain({"question": follow_up_question, "chat_history": chat_history})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "具体的な例を教えてください",
  "chat_history": [
    [
      "LangChain Agentとはなんですか？",
      "LangChain Agentは、OpenAIのLangChainフレームワークを使用して作成された会話モデルです。このエージェントは、対話の内容を記憶し、それを利用して追加の応答を生成することができます。さまざまな種類のメモリを使用して、会話の要約や追加の情報を提供することができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。"
    ]
  ]
}
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "具体的な例を教えてください",
  "chat_history": "\nHuman: LangChain Agentとはなんですか？\nAssistant: LangChain Agentは、OpenAIのLangChainフレームワークを使用して作成された会話モデルです。このエージェントは、対話の内容を記憶し、それを利用して追加の応答を生成することができます。さまざまな種類のメモリを使用して、会話の要約や追加の情報を提供することができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。"
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n次の会話とフォローアップの質問がある場合、会話を考慮してフォローアップの質問を独立した質問に言い換えなさい。\n\n会話:\n\nHuman: LangChain Agentと

In [23]:
res["answer"]

'LangChain Agentは、会話モデルと連携して、対話型のAIチャットボットやアシスタントを作成するために使用されます。このエージェントは、ユーザーとの対話を理解し、適切な応答を生成することができます。また、記憶や要約などの機能を組み合わせて、より高度な対話体験を提供することも可能です。具体的には、会話の記憶や要約、知識グラフの構築など、様々な応用があります。'

### Memoryを使用

In [24]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [25]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["chat_history", "question"]
)

In [26]:
chain_with_memory = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    condense_question_prompt=prompt,
    verbose=True
)

In [27]:
chain_with_memory(question)

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "LangChain Agentとはなんですか？",
  "chat_history": []
}
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "LangChain Agentとはなんですか？",
  "context": "that do things like summarize the conversation as it goes along we've got other ones where we limit the window so it only remembers the last few encounters and then we've got sort of things where you know it merges some of those so where it has the last few but then it does a summary for everything else the summary though is actually done by calling out to a language model itself and asking it hey summarize this conversation so that's something that you will see as well then we've got some sort of more external ways of doing this s

{'question': 'LangChain Agentとはなんですか？',
 'chat_history': [HumanMessage(content='LangChain Agentとはなんですか？'),
  AIMessage(content='LangChain Agentは、OpenAIのLangChainモデルを使用して対話を行うプログラムです。LangChainは、言語モデルを使用して自然な対話を生成するためのフレームワークです。LangChain Agentは、対話のコンテキストを管理し、過去の対話を記憶し、それに基づいて回答を生成することができます。さまざまな種類のメモリを使用して、対話の要約やメモリ管理を行うことができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。')],
 'answer': 'LangChain Agentは、OpenAIのLangChainモデルを使用して対話を行うプログラムです。LangChainは、言語モデルを使用して自然な対話を生成するためのフレームワークです。LangChain Agentは、対話のコンテキストを管理し、過去の対話を記憶し、それに基づいて回答を生成することができます。さまざまな種類のメモリを使用して、対話の要約やメモリ管理を行うことができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。'}

In [28]:
res = chain_with_memory(follow_up_question)

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "具体的な例を教えてください",
  "chat_history": "\nHuman: LangChain Agentとはなんですか？\nAssistant: LangChain Agentは、OpenAIのLangChainモデルを使用して対話を行うプログラムです。LangChainは、言語モデルを使用して自然な対話を生成するためのフレームワークです。LangChain Agentは、対話のコンテキストを管理し、過去の対話を記憶し、それに基づいて回答を生成することができます。さまざまな種類のメモリを使用して、対話の要約やメモリ管理を行うことができます。また、カスタマイズも可能で、独自のメモリシステムを作成することもできます。"
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n次の会話とフォローアップの質問がある場合、会話を考慮してフォローアップの質問を独立した質問に言い換えなさい。\n\n会話:\n\nHuman: LangChain Agentとはなんですか？\nAssistant: LangChain Agentは、OpenAIのLangChainモデルを使用して対話を行うプログラムです。LangChainは、言語モデルを使用して自然な対話を生成するためのフレームワークです。LangChain Agentは、対話のコンテキストを管理し、過去の対話を記憶し、それに基づいて回答を生成することができます。さまざまな種類のメモリを使用して、対話の要約やメモリ管理を行うことができます。また、カスタマイズ

In [29]:
res["answer"]

'LangChain Agentは、様々な方法で回答を生成することができます。これには、会話の要約や過去のエンカウンターの記憶、言語モデルへの問い合わせなどが含まれます。また、カスタマイズすることも可能です。LangChainでは、異なる種類のメモリを使用したり、独自のメモリシステムを作成したりすることができます。'

### デフォルトで使われているプロンプト

In [30]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT

In [31]:
print(CONDENSE_QUESTION_PROMPT.template)


次の会話とフォローアップの質問がある場合、会話を考慮してフォローアップの質問を独立した質問に言い換えなさい。

会話:
{chat_history}
フォローアップの質問: {question}
独立した質問:


In [32]:
print(QA_PROMPT.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
